In [1]:
import pandas as pd
import numpy as np

from itertools import combinations
from scipy import stats

from news_vec.encoder import read_preds

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sns

mpl.style.use('seaborn-muted')
sns.set(style="whitegrid")

In [3]:
df, _ = read_preds('../../data/ava-lstm-attn/')

100%|██████████| 283/283 [00:14<00:00, 19.34it/s]


In [4]:
DOMAINS = sorted(df.domain.unique().tolist())

In [5]:
df['p_true'] = df.apply(lambda r: r[f'p_{r.domain}'], 1)
df['domain_pred'] = df.apply(lambda r: DOMAINS[np.argmax([r[f'p_{d}'] for d in DOMAINS])], 1)

In [6]:
df_test = df[df.split=='test']

In [7]:
rows = []

In [8]:
for d1 in DOMAINS:
    
    d2_count = df_test[df_test.domain==d1].groupby('domain_pred').size().to_dict()
    
    for d2, count in d2_count.items():
        if d2 != d1:
            rows.append((d1, d2, count))

In [13]:
cm_df = pd.DataFrame(rows, columns=('d1', 'd2', 'count'))

In [14]:
cm_df.sort_values('count', ascending=False).head(10)

,d1,d2,count
197,wsj.com,bloomberg.com,362
72,dailycaller.com,breitbart.com,321
98,foxnews.com,apnews.com,275
27,bloomberg.com,wsj.com,267
177,thehill.com,msnbc.com,267
81,dailycaller.com,thehill.com,253
137,msnbc.com,thehill.com,232
64,cnn.com,msnbc.com,232
150,npr.org,nytimes.com,215
67,cnn.com,thehill.com,210


In [15]:
cm_df.to_json('data/hl-graph-lstm-cm.json', orient='records', lines=True)